# Compute orbital Ephemeris workflow

### This script is used to plot the orbital ephemeris of a satellite

In [ ]:
from fds.config import set_api_key, set_url
set_url('https://api.spacetower.exotrail.space/fds/v1')
set_api_key('MY_API_KEY')

In [ ]:
from datetime import datetime, timedelta, UTC
import pandas as pd
from fds.models.orbit_extrapolation.requests import EphemeridesRequest
import plotly.graph_objects as go
from fds.models.orbit_extrapolation.use_case import OrbitExtrapolation
from fds.models.orbital_state import OrbitalState, PropagationContext
from fds.models.spacecraft import SpacecraftSphere
from fds.models.two_line_element import TwoLineElement

In [ ]:
initial_tle = TwoLineElement(
    "1 58295U 23174AR  24137.17541737  .00005775  00000-0  29855-3 0  9994",
    "2 58295  97.4580 211.9750 0010360 352.3995   7.7079 15.16607347 28836"
)

propagation_context = PropagationContext(
    model_perturbations=[
        PropagationContext.Perturbation.DRAG,
        PropagationContext.Perturbation.EARTH_POTENTIAL,
        PropagationContext.Perturbation.SRP,
        PropagationContext.Perturbation.THIRD_BODY,
    ],
    model_solar_flux=150,  # SFU
    model_earth_potential_deg=30,
    model_earth_potential_ord=30,
    model_atmosphere_kind=PropagationContext.AtmosphereModel.HARRIS_PRIESTER,
    integrator_kind=PropagationContext.IntegratorKind.DORMAND_PRINCE_853,
    integrator_min_step=0.01,  # s
    integrator_max_step=100,  # s
)

spacecraft = SpacecraftSphere(
    platform_mass=150,  # kg
    drag_coefficient=2.2,
    cross_section=10,
    reflectivity_coefficient=0.6
)

orbital_state = OrbitalState.from_tle(
    tle=initial_tle,
    propagation_context=propagation_context,
    spacecraft=spacecraft,
)

target_date = datetime.now(UTC) + timedelta(
    seconds=orbital_state.mean_orbit.keplerian_period * 3)

ephemerides_request = EphemeridesRequest(
    ephemeris_types=[EphemeridesRequest.EphemerisType.KEPLERIAN],
    step=200,
)

oe = OrbitExtrapolation.with_target_date(
    target_date=target_date,
    initial_orbital_state=orbital_state,
    ephemerides_request=ephemerides_request,
)

res = oe.run().result

## Plotting the ephemeries

In [ ]:
eph_df = pd.DataFrame(res.export_keplerian_ephemeris())

#  plot
fig = go.Figure()
# fig.add_trace(go.Scatter(x=eph_df['date'], y=eph_df['argument_of_perigee'], mode='lines', name='x'))
# fig.add_trace(go.Scatter(x=eph_df['date'], y=eph_df['eccentricity'], mode='lines', name='y'))
fig.add_trace(go.Scatter(x=eph_df['date'], y=eph_df['semi_major_axis'], mode='lines', name='z'))
# fig.add_trace(go.Scatter(x=eph_df['date'], y=eph_df['raan'], mode='lines', name='z'))
# fig.add_trace(go.Scatter(x=eph_df['date'], y=eph_df['inclination'], mode='lines', name='z'))
fig.update_layout(title='Ephemerides', xaxis_title='Date', yaxis_title='Position [km]')
fig.show()